In [1]:
import sys, os

# 1) 현재 작업 디렉터리(CWD)를 기준으로 삼기
cwd = os.getcwd()  

# 2) 프로젝트 루트까지 올라가기 (예: 노트북이 teleop/utils/datanalysis 폴더에 있다면)
#    폴더 깊이에 맞춰 '..' 개수를 조정하세요
PROJECT_ROOT = os.path.abspath(os.path.join(cwd, '..', '..'))

# 3) sys.path에 삽입
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

    
import numpy as np
import pandas as pd
import h5py
import json
from os.path import join, abspath, dirname
from os import makedirs
from tqdm import tqdm
from PIL import Image
from utils.datanalysis.tactile import split_tactile_data
import cv2

In [2]:
DATA_DIR = ''
EPISODE_DIR = f'episode_0010'
DATA_JSON = 'data.json'

In [3]:
OUTPUT_DIR = 'output'
OUTPUT_DIR = abspath(OUTPUT_DIR)
# Create output directory if it doesn't exist
makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
DATA_JSON_PATH = abspath(join(DATA_DIR, EPISODE_DIR, DATA_JSON))
DATA_JSON_PATH

'/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0010/data.json'

In [5]:
data = json.load(open(DATA_JSON_PATH, 'r'))

info = data['info']
data = data['data']

In [6]:
joint_names = list()

for joint_set_key, joint_set_val in info['joint_names'].items():
    for joint_name in joint_set_val:
        joint_names.append(f'{joint_set_key}.{joint_name}')

joint_names[:5]

['left_arm.kLeftShoulderPitch',
 'left_arm.kLeftShoulderRoll',
 'left_arm.kLeftShoulderYaw',
 'left_arm.kLeftElbow',
 'left_arm.kLeftWristRoll']

In [7]:
proprio_rows = list()
action_rows = list()
rgb_rows = list()
tactile_rows = dict()

for row in tqdm(data[:], desc='Processing rows'):
    joint_row = dict()
    action_row = dict()
    
    for joint_set_key, joint_set_val in info['joint_names'].items():
        
        joint_names = [f'{joint_set_key}.{joint_name}' for joint_name in joint_set_val]
        
        # States for the current joint set
        states = row['states']
        
        for joint_set_key, joint_set_val in info['joint_names'].items():
            
            if (joint_set_key not in states) or (states[joint_set_key]) is None:
                continue
            
            qpos_vals = states[joint_set_key]['qpos']
            qvel_vals = [0.0] * len(joint_set_val)  # Placeholder for qvel, as it's not in the data
  
            
            for joint_name, qpos_val, qvel_val in zip(joint_set_val, qpos_vals, qvel_vals):
                joint_row[f'state.{joint_set_key}.{joint_name}.qpos'] = qpos_val
                joint_row[f'state.{joint_set_key}.{joint_name}.qvel'] = qvel_val

        # Actions for the current joint set
        actions = row['actions']
        
        for joint_set_key, joint_set_val in info['joint_names'].items():
            
            if (joint_set_key not in states) or (states[joint_set_key]) is None:
                continue
            
            qpos_vals = states[joint_set_key]['qpos']
            qvel_vals = [0.0] * len(joint_set_val)  # Placeholder for qvel, as it's not in the data
  
            
            for joint_name, qpos_val, qvel_val in zip(joint_set_val, qpos_vals, qvel_vals):
                action_row[f'action.{joint_set_key}.{joint_name}.qpos'] = qpos_val
                action_row[f'action.{joint_set_key}.{joint_name}.qvel'] = qvel_val
    
    # Tactile data for the current joint set
    tactiles = row['tactiles']
    for tactile_set_key, tactile_set_val in tactiles.items():
        tactile_path = abspath(join(DATA_DIR, EPISODE_DIR, tactile_set_val))
        tactile_npy = np.load(tactile_path)
        tactile_row = tactile_npy.flatten()
        tactile_dict = split_tactile_data(tactile_row)
        
        # add tactile_set_key prefix to each key in tactile_dict
        tactile_dict = {f'tactile.{tactile_set_key}.{k}': v for k, v in tactile_dict.items()}
        
        for key, value in tactile_dict.items():
            if key not in tactile_rows:
                tactile_rows[key] = list()
            tactile_rows[key].append(value)    
    
    camera = row['colors']['color_0']
    rgb_path = abspath(join(DATA_DIR, EPISODE_DIR, camera))
    
    # load jpg and convert to numpy array
    rgb_image = Image.open(rgb_path)
    rgb_image = rgb_image.convert('RGB')
    rgb_array = np.array(rgb_image)
    rgb_rows.append(rgb_array)
    
    proprio_rows.append(joint_row)
    action_rows.append(action_row)
# save npz file with dictionary

# convert to dataframe
proprio_df = pd.DataFrame(proprio_rows)
action_df = pd.DataFrame(action_rows)

# get the primary keys to txt
proprio_keys = proprio_df.columns.tolist()
with open(join(OUTPUT_DIR, f'proprio.txt'), 'w') as f:
    for key in proprio_keys:
        f.write(f'{key}\n')

action_keys = action_df.columns.tolist()
with open(join(OUTPUT_DIR, f'action.txt'), 'w') as f:
    for key in action_keys:
        f.write(f'{key}\n')

# stack the numpy array by looping through the tactile_rows
tactile_rows = {k: np.stack(v) for k, v in tactile_rows.items()}

# 영상 저장용 경로
video_path = join(OUTPUT_DIR, f'{EPISODE_DIR}.mp4')

# assume all RGB frames have same shape
height, width, _ = rgb_rows[0].shape
fps = 30  # or choose based on your time resolution

# OpenCV VideoWriter 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# 프레임을 순차적으로 기록
for frame in rgb_rows:
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR을 사용
    video_writer.write(bgr_frame)

video_writer.release()


npz_dict = {
    'proprio': proprio_df.to_numpy(),
    **tactile_rows,
    'action': action_df.to_numpy()
}

npz_path = join(OUTPUT_DIR, f'{EPISODE_DIR}.npz')
np.savez(npz_path, **npz_dict)

Processing rows: 100%|██████████| 151/151 [00:00<00:00, 438.74it/s]


In [8]:
npz_data = np.load(npz_path, allow_pickle=True)
# Check the keys in the npz file
print(npz_data.files)

['proprio', 'tactile.left_tactile.little_finger_tip', 'tactile.left_tactile.little_finger_nail', 'tactile.left_tactile.little_finger_pad', 'tactile.left_tactile.ring_finger_tip', 'tactile.left_tactile.ring_finger_nail', 'tactile.left_tactile.ring_finger_pad', 'tactile.left_tactile.middle_finger_tip', 'tactile.left_tactile.middle_finger_nail', 'tactile.left_tactile.middle_finger_pad', 'tactile.left_tactile.index_finger_tip', 'tactile.left_tactile.index_finger_nail', 'tactile.left_tactile.index_finger_pad', 'tactile.left_tactile.thumb_tip', 'tactile.left_tactile.thumb_nail', 'tactile.left_tactile.thumb_middle_section', 'tactile.left_tactile.thumb_pad', 'tactile.left_tactile.palm', 'tactile.right_tactile.little_finger_tip', 'tactile.right_tactile.little_finger_nail', 'tactile.right_tactile.little_finger_pad', 'tactile.right_tactile.ring_finger_tip', 'tactile.right_tactile.ring_finger_nail', 'tactile.right_tactile.ring_finger_pad', 'tactile.right_tactile.middle_finger_tip', 'tactile.right_

In [9]:
# Check the shape of the proprio data
proprio_data = npz_data['proprio']
action_data = npz_data['action']
print(f'proprio data shape: {proprio_data.shape}')
print(f'action data shape: {action_data.shape}')
# get the tactile data starts with 'tactile.'
tactile_keys = [key for key in npz_data.files if key.startswith('tactile.')]
# Print the tactile data shapes
for key in tactile_keys:
    tactile_data = npz_data[key]
    print(f'{key} shape: {tactile_data.shape}')

proprio data shape: (151, 52)
action data shape: (151, 52)
tactile.left_tactile.little_finger_tip shape: (151, 3, 3)
tactile.left_tactile.little_finger_nail shape: (151, 12, 8)
tactile.left_tactile.little_finger_pad shape: (151, 10, 8)
tactile.left_tactile.ring_finger_tip shape: (151, 3, 3)
tactile.left_tactile.ring_finger_nail shape: (151, 12, 8)
tactile.left_tactile.ring_finger_pad shape: (151, 10, 8)
tactile.left_tactile.middle_finger_tip shape: (151, 3, 3)
tactile.left_tactile.middle_finger_nail shape: (151, 12, 8)
tactile.left_tactile.middle_finger_pad shape: (151, 10, 8)
tactile.left_tactile.index_finger_tip shape: (151, 3, 3)
tactile.left_tactile.index_finger_nail shape: (151, 12, 8)
tactile.left_tactile.index_finger_pad shape: (151, 10, 8)
tactile.left_tactile.thumb_tip shape: (151, 3, 3)
tactile.left_tactile.thumb_nail shape: (151, 12, 8)
tactile.left_tactile.thumb_middle_section shape: (151, 3, 3)
tactile.left_tactile.thumb_pad shape: (151, 12, 8)
tactile.left_tactile.palm sh